In [ ]:
import sys

sys.path.append("..")

In [ ]:
from pae.loaders.LHCO import ScalarLoaderLHCO, DatasetBuilder

In [ ]:
x = ScalarLoaderLHCO.from_json("../pae/configs/loader/rnd_scalar_2j.json")
mjj = ScalarLoaderLHCO.from_json("../pae/configs/loader/rnd_scalar_mjj.json")

In [ ]:
builder = DatasetBuilder(x, mjj)

In [ ]:
builder.data_preparation(sample_sizes ={'sig':1000, 'bkg': 1_000_000}, fit_key='bkg')



In [ ]:
spec = {'train':{'bkg':1_000_000}, 'test':{'sig':100, 'bkg': 1000}}

In [ ]:
dataset = builder.make_dataset(train = {'bkg':1_000_000}, test={'sig':100, 'bkg': 10_000}, replace=True)

In [ ]:
from pae.density import GMM, ConvKDE
import numpy as np


fftkde = ConvKDE()
fftkde.fit(dataset["mjj_train"])#, range=(1000, 9500)) 
y_kde = fftkde.evaluate(dataset["mjj_train"])

In [ ]:
x_ref = np.linspace(1600, 8000, 1701)

y_kde = fftkde.evaluate(x_ref)

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected = True)
pio.templates.default = "plotly_dark"

In [ ]:
from pae.models.autoencoder import DenseAutoencoder
from pae.models.flows import MAF
from pae.models.nn import PaeBuilder
import tensorflow as tf
import tensorflow.keras as tfk

ae_config = {
    'input_dim':47, 
    'encoding_dim':10, 
    'units':[30, 20, 15],
    'weight_reg':tfk.regularizers.L1L2(l1=1e-5, l2=1e-4),
    'output_activation':tf.nn.sigmoid
}
nf_config = {
    'n_dims':10, 
    'n_layers':5, 
    'units':[32 for _ in range(4)]
}
optimizer_ae = {
    'learning_rate': 0.001
}
optimizer_nf = {
    'learning_rate': 0.005
}

builder = PaeBuilder()
builder.make_ae_model(DenseAutoencoder, ae_config)
builder.make_ae_optimizer(tfk.optimizers.Adam, optimizer_ae)
builder.make_nf_model(MAF, nf_config)
builder.make_nf_optimizer(tfk.optimizers.Adam, optimizer_nf)
builder.compile_ae()
builder.compile_nf()
pae = builder.pae
pae.ae(np.zeros(47).reshape(1,-1))
pae.nf(np.zeros(10).reshape(1,-1))
pae.ae.load_weights("./logs/full-cpu-kde-20211020-165124/ae.h5")
pae.nf.load_weights("./logs/full-cpu-kde-20211020-165124/nf.h5")

In [ ]:
from sklearn.model_selection import KFold

fold5 = KFold(8, shuffle=True)
q= fold5.split(dataset["x_train"])
x_train, x_valid = next(q)
print(x_train.shape)
print(x_valid.shape)

In [ ]:
import tensorflow_probability as tfp
import tqdm
import os
from concurrent.futures import ProcessPoolExecutor 
tfd = tfp.distributions
pae.compute_implicit_sigma(dataset['x_train'][x_valid])
from datetime import datetime
STEPS = 500
BATCH_SIZE = 200

sigma = tf.constant(tf.sqrt(pae.sigma_square))
z_ = tf.Variable(pae.ae.encoder(dataset['x_test']))
opt = tf.optimizers.Adam(learning_rate=0.001)

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
map_summary_writer = tf.summary.create_file_writer(f"./testing/{timestamp}/map")
print(z_.shape)

@tf.function
def max_apriori_prob(x, z, sigma, pae):
    distrs = tfd.MultivariateNormalDiag(loc=x, scale_diag=sigma)
    nf_ll = pae.nf(z)
    reco = pae.ae.decoder(z)
    gauss_ll = distrs.log_prob(reco)
    #tf.print("gauss:", gaussll, "nf:", nfll, "\n")
    return  tf.reduce_mean(-nf_ll - gauss_ll) 


@tf.function
def find_map(x_):
    global z_
    if z_ is None:
        z_ = tf.Variable(pae.ae.encoder(x_))
    z_.assign(pae.ae.encoder(x_))
    for i in range(STEPS):
        with tf.GradientTape() as tape:
            tape.watch(z_)
            nll = max_apriori_prob(x_, z_, sigma, pae)
        grad = tape.gradient(nll, [z_])
        opt.apply_gradients(zip(grad, [z_]))
        with map_summary_writer.as_default():
            tf.summary.scalar('nll', nll, step=i)
    return z_

@tf.function
def tf_graph_map(*args, parallel_iterations=1000):
    return tf.map_fn(*args, parallel_iterations=parallel_iterations)

In [ ]:
ds = tf.convert_to_tensor(dataset['x_test'], dtype=tf.float32)

#ds.shape
# ds = tf.data.Dataset.from_tensor_slices(dataset['x_test'].astype(np.float32))
# ds = ds.cache()
#ds = ds.batch(BATCH_SIZE)
#ds = ds.prefetch()
#print(ds)

In [ ]:
%%time
tf.profiler.experimental.start(f"./testing/{timestamp}")
with tf.device("GPU:0"):
    # ta = tf.TensorArray(tf.float32, size=0, dynamic_size=True)
    # for i, x in enumerate(ds):
    #     ta.write(i, find_map(x))
    # z_map = ta.concat()
    # ta.close()
    z_map = find_map(ds)
tf.profiler.experimental.stop()
#     z_map = tf_graph_map(find_map, x_test, parallel_iterations=1000)

In [ ]:
print(z_map.shape)

In [ ]:
print(z_map[0])